<a href="https://colab.research.google.com/github/SiDDyy007/Phishing_Websites_Detector_Using_KNN/blob/main/KNN_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
# data = pd.read_csv("/content/phishing.csv")
data = pd.read_csv('/content/phishing.csv', nrows=1000)
data = data.drop(['Index'], axis=1)
data.head()
    

,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,Favicon,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
0,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,-1,0,-1,1,-1,-1,1,1,-1,1,...,1,1,1,1,1,-1,1,-1,-1,1


In [109]:
X = data.drop(columns="class")

Y= data["class"]
# Y.head()

In [110]:
# Calculate distance between two points

def minkowski_distance(a, b, p=1):
    
    # Store the number of dimensions
    dim = len(a)
    
    # Set initial distance to 0
    distance = 0
    
    # Calculate minkowski distance using parameter p
    for d in range(dim):
        distance += abs(a[d] - b[d])**p
        
    distance = distance**(1/p)
    
    return distance


# Test the function

minkowski_distance(a=X.iloc[0], b=X.iloc[1], p=1)

14.0

In [111]:
#Computes cosine similarity between the two vectors

def cosine_distance(a,b):
       
    distance=1-np.dot(a,b)/np.sqrt(np.dot(a,a)*np.dot(b,b))
    return distance

In [112]:
def jaccard_similarity(A, B):
    #Find intersection of two sets
    A = set(A)
    b = set(B)
    nominator = A.intersection(B)

    #Find union of two sets
    denominator = A.union(B)

    #Take the ratio of sizes
    similarity = len(nominator)/len(denominator)
    
    return similarity


In [113]:
# Define an arbitrary test point

test_pt = [4.8, 2.7, 2.5, 0.7]

# Calculate distance between test_pt and all points in X

distances = []

for i in X.index:
    
    distances.append(minkowski_distance(test_pt, X.iloc[i]))
    
df_dists = pd.DataFrame(data=distances, index=X.index, columns=['dist'])
df_dists.head()

,dist
0,7.3
1,8.3
2,8.3
3,10.3
4,12.3


In [114]:

# Find the 5 nearest neighbors

df_nn = df_dists.sort_values(by=['dist'], axis=0)[:5]
df_nn

,dist
0,7.3
778,7.3
775,7.3
773,7.3
245,7.3


In [115]:

from collections import Counter

# Create counter object to track the labels

counter = Counter(Y[df_nn.index])

# Get most common label of all the nearest neighbors

counter.most_common()[0][0]

-1

In [127]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import TSNE

# Split the data - 75% train, 25% test

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25,
                                                   random_state=1)

# Scale the X data


# scaler = OneHotEncoder() // Not working
# scaler = StandardScaler()
# scaler = MinMaxScaler()
# scaler = PCA(n_components='mle')
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
# ---------------------
scaler = MDS()
# scalar = TSNE()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)



In [128]:
def knn_predict(X_train, X_test, y_train, y_test, k, p):
    
    # Counter to help with label voting
    from collections import Counter
    
    # Make predictions on the test data
    # Need output of 1 prediction per test data point
    y_hat_test = []

    for test_point in X_test:
        distances = []

        for train_point in X_train:
            distance = minkowski_distance(test_point, train_point, p=p)
            distances.append(distance)
        
        # Store distances in a dataframe
        df_dists = pd.DataFrame(data=distances, columns=['dist'], 
                                index=y_train.index)
        
        # Sort distances, and only consider the k closest points
        df_nn = df_dists.sort_values(by=['dist'], axis=0)[:k]

        # Create counter object to track the labels of k closest neighbors
        counter = Counter(y_train[df_nn.index])

        # Get most common label of all the nearest neighbors
        prediction = counter.most_common()[0][0]
        
        # Append prediction to output list
        y_hat_test.append(prediction)
        
    return y_hat_test


# Make predictions on test dataset


# print(y_hat_test)

In [129]:
y_hat_test = knn_predict(X_train, X_test, y_train, y_test, k=5, p=1)

In [119]:
y_hat_test_2 = knn_predict(X_train, X_test, y_train, y_test, k=3, p=2)

In [120]:
y_hat_test_3 = knn_predict(X_train, X_test, y_train, y_test, k=3, p=10000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in double_scalars
  del sys.path[0]


In [121]:
def knn_predict(X_train, X_test, y_train, y_test, k, p):
    
    # Counter to help with label voting
    from collections import Counter
    
    # Make predictions on the test data
    # Need output of 1 prediction per test data point
    y_hat_test = []

    for test_point in X_test:
        distances = []

        for train_point in X_train:
            distance = cosine_distance(test_point, train_point)
            distances.append(distance)
        
        # Store distances in a dataframe
        df_dists = pd.DataFrame(data=distances, columns=['dist'], 
                                index=y_train.index)
        
        # Sort distances, and only consider the k closest points
        df_nn = df_dists.sort_values(by=['dist'], axis=0)[:k]

        # Create counter object to track the labels of k closest neighbors
        counter = Counter(y_train[df_nn.index])

        # Get most common label of all the nearest neighbors
        prediction = counter.most_common()[0][0]
        
        # Append prediction to output list
        y_hat_test.append(prediction)
        
    return y_hat_test


# Make predictions on test dataset


# print(y_hat_test)

In [122]:
cosine_test = knn_predict(X_train, X_test, y_train, y_test, k=5, p=1)
cosine_test_2 = knn_predict(X_train, X_test, y_train, y_test, k=3, p=2)

In [123]:
def knn_predict(X_train, X_test, y_train, y_test, k, p):
    
    # Counter to help with label voting
    from collections import Counter
    
    # Make predictions on the test data
    # Need output of 1 prediction per test data point
    y_hat_test = []

    for test_point in X_test:
        distances = []

        for train_point in X_train:
            distance = jaccard_similarity(test_point, train_point)
            distances.append(distance)
        
        # Store distances in a dataframe
        df_dists = pd.DataFrame(data=distances, columns=['dist'], 
                                index=y_train.index)
        
        # Sort distances, and only consider the k closest points
        df_nn = df_dists.sort_values(by=['dist'], axis=0)[:k]

        # Create counter object to track the labels of k closest neighbors
        counter = Counter(y_train[df_nn.index])

        # Get most common label of all the nearest neighbors
        prediction = counter.most_common()[0][0]
        
        # Append prediction to output list
        y_hat_test.append(prediction)
        
    return y_hat_test


# Make predictions on test dataset


# print(y_hat_test)

In [124]:
jaccard_test = knn_predict(X_train, X_test, y_train, y_test, k=5, p=1)
jaccard_test_2 = knn_predict(X_train, X_test, y_train, y_test, k=3, p=2)

In [130]:

# Get test accuracy score

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_hat_test))
print(accuracy_score(y_test, y_hat_test_2))
print(accuracy_score(y_test, y_hat_test_3))
print(accuracy_score(y_test, cosine_test))
print(accuracy_score(y_test, cosine_test_2))
print(accuracy_score(y_test, jaccard_test))
print(accuracy_score(y_test, jaccard_test_2))

0.612
0.712
0.748
0.724
0.696
0.556
0.556


In [126]:
# Testing to see results from sklearn.neighbors.KNeighborsClassifier

from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=5, p=1)
clf.fit(X_train, y_train)
y_pred_test = clf.predict(X_test)

clf2 = KNeighborsClassifier(n_neighbors=3, p=2)
clf2.fit(X_train, y_train)
y_pred_test2 = clf2.predict(X_test)

print(f"Sklearn KNN Accuracy: {accuracy_score(y_test, y_pred_test)}")
print(f"Sklearn KNN Accuracy: {accuracy_score(y_test, y_pred_test2)}")

Sklearn KNN Accuracy: 0.708
Sklearn KNN Accuracy: 0.712
